# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_excertos.CSV', sep = ';')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_EXCERTO,TEXTO_EXCERTO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995438,Voto:Cuidam os autos de tomada de contas espec...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025603,Voto:Cuidam os autos de Solicitação do Congres...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455375,Relatório:Trata-se de embargos de declaração o...,Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773747,Voto:8. Em relação a outros processos judiciai...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773403,Voto:11. Relativamente ao ato envolvendo a Sra...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13285, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          553
Contrato Administrativo     941
Convênio                    683
Desestatização              139
Direito Processual         1811
Finanças Públicas           328
Gestão Administrativa       338
Licitação                  2756
Pessoal                    3393
Responsabilidade           2343
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13285, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 2000
dim_vetor = 50

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_EXCERTO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_EXCERTO'])

Using TensorFlow backend.


Found 63925 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13285, 2000)


In [9]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('../externos/model-50.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 50))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 63925
Encontrados no modelo: 42309 = 66.18537348455222


# Treinamento

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1109 21:38:39.397022 139841005438784 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1109 21:38:39.508047 139841005438784 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1109 21:38:39.510013 139841005438784 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1109 21:38:39.517586 139841005438784 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 3,434,646
Trainable params: 238,346
Non-trainable params: 3,196,300
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 555s 52ms/step - loss: 1.5679 - categorical_accuracy: 0.4743 - val_loss: 1.6392 - val_categorical_accuracy: 0.4592
Epoch 2/20
10628/10628 [==============================] - 532s 50ms/step - loss: 1.1560 - categorical_accuracy: 0.6139 - val_loss: 2.0258 - val_categorical_accuracy: 0.

In [12]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 597s 56ms/step - loss: 1.5100 - categorical_accuracy: 0.4985 - val_loss: 1.4381 - val_categorical_accuracy: 0.5499
Epoch 2/20
10628/10628 [==============================] - 597s 56ms/step - loss: 0.8623 - categorical_accuracy: 0.7068 - val_loss: 0.9596 - val_categorical_accuracy: 0.6767
E

In [13]:
from keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(SimpleRNN(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               78592     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 3,277,462
Trainable params: 3,277,462
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 207s 20ms/step - loss: 1.8022 - categorical_accuracy: 0.3792 - val_loss: 1.8472 - val_categorical_accuracy: 0.3632
Epoch 2/20
10628/10628 [==============================] - 207s 19ms/step - loss: 1.5877 - categorical_accuracy: 0.4645 - val_loss: 1.7761 - val_categorical_accuracy: 0.3959
E

In [14]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               314368    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 3,513,238
Trainable params: 3,513,238
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 690s 65ms/step - loss: 1.6297 - categorical_accuracy: 0.4571 - val_loss: 1.6669 - val_categorical_accuracy: 0.3948
Epoch 2/20
10628/10628 [==============================] - 689s 65ms/step - loss: 1.4385 - categorical_accuracy: 0.5368 - val_loss: 1.6570 - val_categorical_accuracy: 0.4373
E

In [15]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(SimpleRNN(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               157184    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 3,358,614
Trainable params: 3,358,614
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 318s 30ms/step - loss: 1.8694 - categorical_accuracy: 0.3506 - val_loss: 2.0536 - val_categorical_accuracy: 0.1607
Epoch 2/20
10628/10628 [==============================] - 318s 30ms/step - loss: 1.7790 - categorical_accuracy: 0.3896 - val_loss: 1.9545 - val_categorical_accuracy: 0.3150
E

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(GRU(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 3,672,982
Trainable params: 3,672,982
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 960s 90ms/step - loss: 1.5467 - categorical_accuracy: 0.4857 - val_loss: 1.3788 - val_categorical_accuracy: 0.5250
Epoch 2/20
10628/10628 [==============================] - 960s 90ms/step - loss: 0.8882 - categorical_accuracy: 0.7022 - val_loss: 1.1880 - val_categorical_accuracy: 0.5713
E

In [13]:
from keras.layers import Bidirectional, LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1111 10:31:41.187237 140316744189760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1111 10:31:41.314905 140316744189760 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               628736    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 3,830,166
Trainable params: 3,830,166
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1299s 122ms/step - loss: 1.6197 - categorical_accuracy: 0.4676 - val_loss: 1.6418 - val_categorical_accuracy: 0.4603
Epoch 2/20
10628/10628 [==============================] - 1284s 121ms/step - loss: 1.2963 - categorical_accuracy: 0.5903 - val_loss: 1.2863 - val_categorical_accuracy: 0.57

In [14]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(128))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               68736     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 3,266,326
Trainable params: 3,266,326
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 555s 52ms/step - loss: 1.6034 - categorical_accuracy: 0.4658 - val_loss: 2.5983 - val_categorical_accuracy: 0.3192
Epoch 2/20
10628/10628 [==============================] - 555s 52ms/step - loss: 1.0397 - categorical_accuracy: 0.6543 - val_loss: 1.4482 - val_categorical_accuracy: 0.5551
E

In [15]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(64))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                22080     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 3,219,030
Trainable params: 3,219,030
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 549s 52ms/step - loss: 1.6525 - categorical_accuracy: 0.4384 - val_loss: 2.1563 - val_categorical_accuracy: 0.3681
Epoch 2/20
10628/10628 [==============================] - 548s 52ms/step - loss: 1.1358 - categorical_accuracy: 0.6306 - val_loss: 1.4984 - val_categorical_accuracy: 0.5171
E

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_3 (GRU)                  (None, 512)               864768    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 4,066,198
Trainable params: 4,066,198
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 549s 52ms/step - loss: 1.6841 - categorical_accuracy: 0.4659 - val_loss: 1.2552 - val_categorical_accuracy: 0.5947
Epoch 2/20
10628/10628 [==============================] - 549s 52ms/step - loss: 0.8227 - categorical_accuracy: 0.7266 - val_loss: 1.0896 - val_categorical_accuracy: 0.6394
E

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_4 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
Total params: 3,466,262
Trainable params: 3,466,262
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 548s 52ms/step - loss: 1.5363 - categorical_accuracy: 0.4850 - val_loss: 1.7790 - val_categorical_accuracy: 0.5047
Epoch 2/20
10628/10628 [==================

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_5 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_7 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                650       
Total params: 3,449,174
Trainable params: 3,449,174
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 545s 51ms/step - loss: 1.5458 - categorical_accuracy: 0.4850 - val_loss: 1.4242 - val_categorical_accuracy: 0.5265
Epoch 2/20
10628/10628 [==================

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_6 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_9 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                330       
Total params: 3,440,630
Trainable params: 3,440,630
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 551s 52ms/step - loss: 1.5851 - categorical_accuracy: 0.4690 - val_loss: 1.6176 - val_categorical_accuracy: 0.4780
Epoch 2/20
10628/10628 [==================

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1112 11:54:13.561027 140316744189760 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 645s 61ms/step - loss: 1.6276 - categorical_accuracy: 0.4529 - val_loss: 1.5301 - val_categorical_accuracy: 0.4900
Epoch 2/20
10628/10628 [==============================] - 642s 60ms/step - loss: 1.0676 - categorical_accuracy: 0.6519 - val_loss: 1.0917 - val_categorical_accuracy: 0.6394
E

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_8 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_12 (Dense)             (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 638s 60ms/step - loss: 1.7145 - categorical_accuracy: 0.4172 - val_loss: 1.6609 - val_categorical_accuracy: 0.4219
Epoch 2/20
10628/10628 [==============================] - 636s 60ms/step - loss: 1.1842 - categorical_accuracy: 0.6155 - val_loss: 1.1963 - val_categorical_accuracy: 0.5864
E

In [22]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_9 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_13 (Dense)             (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 561s 53ms/step - loss: 1.4895 - categorical_accuracy: 0.4989 - val_loss: 1.2339 - val_categorical_accuracy: 0.6214
Epoch 2/20
10628/10628 [==============================] - 559s 53ms/step - loss: 0.8223 - categorical_accuracy: 0.7224 - val_loss: 1.0568 - val_categorical_accuracy: 0.6650
E

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_10 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 557s 52ms/step - loss: 1.6728 - categorical_accuracy: 0.4335 - val_loss: 1.6345 - val_categorical_accuracy: 0.4185
Epoch 2/20
10628/10628 [==============================] - 555s 52ms/step - loss: 1.1424 - categorical_accuracy: 0.6219 - val_loss: 1.3226 - val_categorical_accuracy: 0.5386
E

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, return_sequences=True))
model.add(GRU(32))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_11 (GRU)                 (None, 2000, 256)         235776    
_________________________________________________________________
gru_12 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                330       
Total params: 3,460,150
Trainable params: 3,460,150
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1131s 106ms/step - loss: 1.4902 - categorical_accuracy: 0.4984 - val_loss: 1.8489 - val_categorical_accuracy: 0.4569
Epoch 2/20
10628/10628 [================

In [25]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_13 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_16 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1290      
Total params: 3,466,262
Trainable params: 3,466,262
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 655s 62ms/step - loss: 1.

In [27]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_15 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 655s 62ms/step - loss: 1.6689 - categorical_accuracy: 0.4357 - val_loss: 1.4953 - val_categorical_accuracy: 0.4806
Epoch 2/20
10628/10628 [==============================] - 654s 61ms/step - loss: 1.0669 - categorical_accuracy: 0.6465 - val_loss: 1.0863 - val_categorical_accuracy: 0.6432
E

In [28]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(GRU(256, dropout=0.1, recurrent_dropout=0.4)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                5130      
Total params: 3,672,982
Trainable params: 3,672,982
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1275s 120ms/step - loss: 1.6524 - categorical_accuracy: 0.4453 - val_loss: 1.7721 - val_categorical_accuracy: 0.4008
Epoch 2/20
10628/10628 [==============================] - 1272s 120ms/step - loss: 1.0856 - categorical_accuracy: 0.6421 - val_loss: 1.1439 - val_categorical_accuracy: 0.63

In [29]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_17 (GRU)                 (None, 512)               864768    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 4,066,198
Trainable params: 4,066,198
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 690s 65ms/step - loss: 1.6559 - categorical_accuracy: 0.4474 - val_loss: 1.4306 - val_categorical_accuracy: 0.5344
Epoch 2/20
10628/10628 [==============================] - 688s 65ms/step - loss: 1.1013 - categorical_accuracy: 0.6434 - val_loss: 1.3003 - val_categorical_accuracy: 0.5924
E

In [30]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(1024, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_18 (GRU)                 (None, 1024)              3302400   
_________________________________________________________________
dense_22 (Dense)             (None, 10)                10250     
Total params: 6,508,950
Trainable params: 6,508,950
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 719s 68ms/step - loss: 1.9008 - categorical_accuracy: 0.3542 - val_loss: 2.0284 - val_categorical_accuracy: 0.2902
Epoch 2/20
10628/10628 [==============================] - 715s 67ms/step - loss: 1.9664 - categorical_accuracy: 0.3107 - val_loss: 2.8366 - val_categorical_accuracy: 0.2172
E